In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from hda import Client
import os
import datetime
import time
import matplotlib
import cfgrib

In [2]:
c = Client(debug=True)
month = "0" + str(1)
print(month)

01


In [3]:
data = {
  "datasetId": "EO:ECMWF:DAT:REANALYSIS_ERA5_LAND",
  "boundingBoxValues": [
    {
      "name": "area",
      "bbox": [
        4.919044320039164,
        45.73676238441797,
        4.919044320039165,
        45.73676238441798,
      ]
    }
  ],
  "multiStringSelectValues": [
    {
      "name": "day",
      "value": [
        "10",
        "11",
        "12",
        "13",
        "14",
        "15",
        "16",
        "17",
        "18",
        "19",
        "20",
        "21",
        "22",
        "23",
        "24",
        "25",
        "26",
        "27",
        "28",
        "29",
        "30",
        "31",
        "01",
        "02",
        "03",
        "04",
        "05",
        "06",
        "07",
        "08",
        "09"
      ]
    },
    {
      "name": "time",
      "value": [
        "00:00",
        "01:00",
        "02:00",
        "03:00",
        "04:00",
        "05:00",
        "06:00",
        "07:00",
        "08:00",
        "09:00",
        "10:00",
        "11:00",
        "12:00",
        "13:00",
        "14:00",
        "15:00",
        "16:00",
        "17:00",
        "18:00",
        "19:00",
        "20:00",
        "21:00",
        "22:00",
        "23:00"
      ]
    },
    {
      "name": "variable",
      "value": [
        "2m_temperature"
      ]
    }
  ],
  "stringChoiceValues": [
    {
      "name": "format",
      "value": "netcdf"
    },
    {
      "name": "month",
      "value": "01"
    },
    {
      "name": "year",
      "value": "2022"
    }
  ]
}

In [ ]:
stamp = time.time()
datasets = []

dates = ['01','02','03','04','05','06','07','08','09','10','11','12']

for i in dates:
    data['stringChoiceValues'][1]['value'] = i
    matches = c.search(data)
    matches.download()
    
    for match in matches.results:
        fdst = match['filename']
        print(f"Found: {fdst}")
        era5_ds = xr.open_dataset(fdst)
        xr.decode_cf(era5_ds)
        datasets.append(era5_ds)

#filename = '2m_temperatures_1h_lyon' + str(int(stamp)) + '.nc'

#os.rename(fdst, filename)

Found: adaptor.mars.internal-1686496765.3737743-19693-9-d47ed260-17d3-4dbe-9da7-1c427c4b17d3.nc



  0%|                                                                                      | 0.00/4.99k [00:00<?, ?B/s]
                                                                                                                       

Found: adaptor.mars.internal-1686506323.3676383-2061-15-0cd439fe-cc90-4449-b4ed-ca84bd5725c7.nc


In [ ]:
#era5_ds = xr.open_dataset(filename)
#xr.decode_cf(era5_ds)

#era5_ds = xr.open_dataset(filename, engine="cfgrib")

era5_ds = xr.concat(datasets, dim='time')

In [ ]:
print(era5_ds)

lats = era5_ds['latitude']
print("Latitude South: {0:.2f}, Latitude North: {1:.2f}".format(lats.data.min(), lats.data.max()))

lons = era5_ds['longitude']
print("Latitude West: {0:.2f}, Latitude East: {1:.2f}".format(lons.data.min(), lons.data.max()))

In [ ]:
temperature2m = era5_ds['t2m']
print(temperature2m)

In [ ]:
print(temperature2m.values)

In [ ]:
print(temperature2m.coords)

In [ ]:
print(temperature2m.coords['longitude'].values)

In [ ]:
print(temperature2m.coords['latitude'].values)

In [ ]:
print(temperature2m.coords['time'].values)

In [ ]:
print(temperature2m.attrs)

In [ ]:
print(temperature2m.attrs['units'])

In [ ]:
print(temperature2m.attrs['long_name'])

In [ ]:
for i in range (0, 1):
    (temperature2m[i, :, :]-273.15).plot(cmap = 'jet')
    time.sleep(1)

In [ ]:
temperature2m.mean() - 273.15

In [ ]:
(temperature2m.sel(time = '2022-05-16T14:00:00.000000000')-273.15).plot(cmap = 'jet')

In [ ]:
temperature2m

In [ ]:
"""
finding frost days: 
"""

daily_mean_temps = temperature2m.min(dim=['longitude', 'latitude'])

#print(daily_mean_temps)

daily_temps = daily_mean_temps.resample(time='D').min()    #taking min values of temperature for the region in an array

under0_days = daily_temps.where(daily_temps.values < 273.15)    #extract array with days where min temp was under 0
under0_days = under0_days.dropna(dim='time')

above2_days = daily_temps.where(daily_temps.values > 275)      #extract array with days where min temp was above 2
above2_days = above2_days.dropna(dim = 'time')

#print(daily_temps)
#print(under0_days)

frost_array = []
for i in under0_days.time:
    frost_array.append(i.values)       #append the time values of the days where there was frost


In [ ]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']



frost_first_half = [date for date in frost_array if (pd.to_datetime(date).month) < 7]     #cut year in 2 bf4 july to find seas
frost_2nd_half = [date for date in frost_array if (pd.to_datetime(date).month) >= 7]     #cut year in 2 after july


first_frost_day = pd.to_datetime(min(frost_2nd_half))
last_frost_day = pd.to_datetime(max(frost_first_half))

#do the same for above2_days




#print(first_frost_day)
#print(last_frost_day)

print(f"In {first_frost_day.year}, the last frost day happened on {last_frost_day}, and frost came back on {first_frost_day}. The growing season of this place is hence between {months[last_frost_day.month - 1]} and {months[first_frost_day.month - 1]}.")

In [ ]:
"""
Calculate median temperature over growing season:
"""
print(last_frost_day, first_frost_day, type(last_frost_day))

last_frost_day = last_frost_day + datetime.timedelta(days=1)
first_frost_day = first_frost_day - datetime.timedelta(days=1)

last_frost_day = datetime.datetime(last_frost_day.year, last_frost_day.month, last_frost_day.day)
first_frost_day = datetime.datetime(first_frost_day.year, first_frost_day.month, first_frost_day.day)


las_tmsp = pd.to_datetime(last_frost_day)
first_tmsp = pd.to_datetime(first_frost_day)

print(las_tmsp, first_tmsp)



mask = (temperature2m.time > las_tmsp) & (temperature2m.time < first_tmsp)

seasons_temps = temperature2m.sel(time=mask)      #array with temperatures of the growing season

print(seasons_temps)

In [ ]:
print(seasons_temps.values.min())

In [ ]:
#create array with median daily temperatures

daily_mean_temps = seasons_temps.mean(dim=['longitude', 'latitude']) - 273.15

# Resample the daily mean temperatures to a daily frequency
daily_temps = daily_mean_temps.resample(time='D').mean()


daily_temps

print(daily_temps.min(), daily_temps.max())

In [ ]:
average_min_temp_season = daily_temps.values.min()
average_max_temp_season = daily_temps.values.max()

print(average_min_temp_season, average_max_temp_season)